# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [13]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [14]:
from sklearn.model_selection import train_test_split
X = adult_dt.drop(columns="income") 
Y = adult_dt['income']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?




random_state in the splitting function initialize the random number generator that controls various processes in the algorithm - any randomization in the decision-making process during the training of models
Setting a fixed value for the random state ensures that the random processes in the model is reproducible
Evaluating models using cross-validation, the same random state ensures that the data is split in exactly the same way every code run

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

numerical_features = [
    "age", "fnlwgt", "education-num", "capital-gain", 
    "capital-loss", "hours-per-week"
]
categorical_features = [
    "workclass", "education", "marital-status", "occupation", 
    "relationship", "race", "sex", "native-country"
]

numerical_transformer = Pipeline(steps=[
    ("knn_imputer", KNNImputer(n_neighbors=7, weights="distance")),
    ("scaler", RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ("simple_imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", drop="if_binary"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num_transforms", numerical_transformer, numerical_features),
        ("cat_transforms", categorical_transformer, categorical_features)
    ]
)

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [16]:
from sklearn.ensemble import RandomForestClassifier

model_pipeline = Pipeline(steps=[
    ("preprocessing", preprocessor),    
    ("classifier", RandomForestClassifier())  
])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [18]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score

scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

cv_results = cross_validate(
    model_pipeline, 
    X_train, 
    Y_train, 
    cv=5, 
    scoring=scoring, 
    return_train_score=True
)

print("The training and validation results:")
for metric in scoring.keys():
    train_scores = cv_results[f'train_{metric}']
    test_scores = cv_results[f'test_{metric}']
    
    print(f"\nMetric: {metric}")
    print(f"  Training scores (per fold): {train_scores}")
    print(f"  Validation scores (per fold): {test_scores}")
    print(f"  Average training score: {train_scores.mean():.4f}")
    print(f"  Average validation score: {test_scores.mean():.4f}")

c:\Users\alyon\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\alyon\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


The training and validation results:

Metric: neg_log_loss
  Training scores (per fold): [-0.08162018 -0.0812617  -0.0812555  -0.08145621 -0.08107471]
  Validation scores (per fold): [-0.35609692 -0.36095817 -0.39781856 -0.35798222 -0.37339627]
  Average training score: -0.0813
  Average validation score: -0.3693

Metric: roc_auc
  Training scores (per fold): [1.         1.         1.         0.99999997 1.        ]
  Validation scores (per fold): [0.9047839  0.90193644 0.90109027 0.90624757 0.90252911]
  Average training score: 1.0000
  Average validation score: 0.9033

Metric: accuracy
  Training scores (per fold): [1.         1.         1.         0.99994516 1.        ]
  Validation scores (per fold): [0.85018644 0.84558017 0.85256692 0.86024572 0.85585783]
  Average training score: 1.0000
  Average validation score: 0.8529

Metric: balanced_accuracy
  Training scores (per fold): [1.        1.        1.        0.9999638 1.       ]
  Validation scores (per fold): [0.77296559 0.7643937

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [19]:
results_df = pd.DataFrame(cv_results)
sorted_results_df = results_df.sort_values(by='test_neg_log_loss')
sorted_results_df

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
2,7.876616,0.163558,-0.397819,-0.081256,0.901090,1.0,0.852567,1.000000,0.774080,1.000000
4,7.872824,0.156918,-0.373396,-0.081075,0.902529,1.0,0.855858,1.000000,0.778098,1.000000
1,8.032861,0.171162,-0.360958,-0.081262,0.901936,1.0,0.845580,1.000000,0.764394,1.000000
3,7.944032,0.166118,-0.357982,-0.081456,0.906248,1.0,0.860246,0.999945,0.783148,0.999964
0,7.890917,0.167167,-0.356097,-0.081620,0.904784,1.0,0.850186,1.000000,0.772966,1.000000


Calculate the mean of each metric. 

In [20]:
mean_results = sorted_results_df.mean()

print("Mean results for each metric:")
print(mean_results)

Mean results for each metric:
fit_time                   7.923450
score_time                 0.164985
test_neg_log_loss         -0.369250
train_neg_log_loss        -0.081334
test_roc_auc               0.903317
train_roc_auc              1.000000
test_accuracy              0.852887
train_accuracy             0.999989
test_balanced_accuracy     0.774537
train_balanced_accuracy    0.999993
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [23]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

model_pipeline.fit(X_train, Y_train)

Y_pred = model_pipeline.predict(X_test)
Y_pred_proba = model_pipeline.predict_proba(X_test)

test_metrics = {
    "neg_log_loss": log_loss(Y_test, Y_pred_proba),
    "roc_auc": roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    "accuracy": accuracy_score(Y_test, Y_pred),
    "balanced_accuracy": balanced_accuracy_score(Y_test, Y_pred)
}

test_metrics

{'neg_log_loss': 0.38478347470623997,
 'roc_auc': 0.9000414761690279,
 'accuracy': 0.8540280479066434,
 'balanced_accuracy': 0.7736803709030328}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

this can help us to further sort the dataset, although its not clear if the program going to run faster 

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.